In [0]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 195.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 MB 222.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 200.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
def widget(name, default):
    try:
        if value:= dbutils.widgets.get(name):
            return value
    except Exception:
        pass
    dbutils.widgets.text(name, default)
    return default

CATALOG_NAME = widget("CATALOG_NAME", "iot_dev")
SCHEMA_NAME  = widget("SCHEMA_NAME", "iot_ingest")
MODEL_ID     = widget("MODEL_ID", "yolo_object_detect")
ALIAS        = widget("ALIAS", "champion")
REGISTERED_MODEL_NAME = f"{CATALOG_NAME}.{SCHEMA_NAME}.{MODEL_ID}"
EXPERIMENT_PATH = f"/Shared/{REGISTERED_MODEL_NAME}"


In [0]:
import mlflow.pyfunc
from typing import Any

import mlflow.pyfunc
from typing import Any


class YoloDetectionModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import os
        import tempfile
        from pathlib import Path

        yolo_config_dir = (
            Path(tempfile.gettempdir())
            / CATALOG_NAME
            / SCHEMA_NAME
            / MODEL_ID
            / "yolo_config"
        )
        yolo_config_dir.mkdir(parents=True, exist_ok=True)
        os.environ["YOLO_CONFIG_DIR"] = str(yolo_config_dir)

        from ultralytics import YOLO

        self.model = YOLO(context.artifacts["weights"])
        self.class_names = self.model.names  # id -> label

    def _decode(self, b64: str):
        import base64
        import numpy as np
        import cv2

        arr = np.frombuffer(base64.b64decode(b64), dtype=np.uint8)
        return cv2.imdecode(arr, cv2.IMREAD_COLOR)

    def predict(self, context, df):
        import pandas as pd

        images = [self._decode(b) for b in df["image_base64"]]
        results = self.model(images, conf=0.25, batch=len(images))

        rows: list[dict[str, Any]] = []
        for r in results:
            boxes = []
            if r.boxes is not None:
                xyxy = r.boxes.xyxy.cpu().numpy()
                conf = r.boxes.conf.cpu().numpy()
                cls = r.boxes.cls.cpu().numpy()

                for i in range(len(xyxy)):
                    class_id = int(cls[i])
                    boxes.append(
                        {
                            "label": self.class_names[class_id],
                            "confidence": float(conf[i]),
                            "x1": float(xyxy[i][0]),
                            "y1": float(xyxy[i][1]),
                            "x2": float(xyxy[i][2]),
                            "y2": float(xyxy[i][3]),
                        }
                    )

            rows.append({"boxes": boxes})

        return pd.DataFrame(rows)

/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:155: FutureWarning: Model's `predict` method contains invalid parameters: {'df'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [0]:
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec, DataType


import io
import base64
import pandas as pd
from PIL import Image
import mlflow


input_schema = Schema([ColSpec(DataType.string, "image_base64")])
output_schema = Schema([ColSpec(DataType.string, "json_result")])

signature = ModelSignature(
    inputs=input_schema,
    outputs=output_schema,
)

# create a tiny valid base64 image for input_example
buf = io.BytesIO()
Image.new("RGB", (2, 2), (255, 255, 255)).save(buf, format="PNG")
png_bytes = buf.getvalue()
image_base_64 = base64.b64encode(png_bytes).decode("utf-8")

# validate base64 image
Image.open(io.BytesIO(base64.b64decode(image_base_64))).verify()

input_example = pd.DataFrame({"image_base64": [image_base_64]})

mlflow.set_experiment(EXPERIMENT_PATH)

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=YoloDetectionModel(),
        artifacts={
            "weights": "yolo11n.pt",
        },
        pip_requirements=["ultralytics"],
        input_example=input_example,
        signature=signature,
    )

model_uri = f"runs:/{run.info.run_id}/model"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/iot_dev/iot_ingest/yolo_object_detect/yolo_config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


2025/12/17 03:08:47 INFO mlflow.pyfunc: Validating input example against model signature



0: 640x640 (no detections), 567.9ms
Speed: 14.3ms preprocess, 567.9ms inference, 22.2ms postprocess per image at shape (1, 3, 640, 640)


In [0]:
from mlflow.tracking import MlflowClient

mv = mlflow.register_model(
    model_uri=model_uri,
    name=REGISTERED_MODEL_NAME,
)

MlflowClient().set_registered_model_alias(
    name=REGISTERED_MODEL_NAME,
    alias=ALIAS,
    version=mv.version,
)

print(mv.version)

Registered model 'iot_dev.iot_ingest.yolo_object_detect' already exists. Creating a new version of this model...
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://dlsentdatadbrdeveastus1.dfs.core.windows.net/entdata-test/models%2F9907718f-d1e3-410a-bc94-fe804b8d6e9e%2Fversions%2Fb980ac74-861a-409f-bcdf-b7f5136b00ec%2Fpython_env.yaml?resource=REDACTED&st=REDACTED&sv=REDACTED&ske=REDACTED&sig=REDACTED&sktid=REDACTED&se=REDACTED&sdd=REDACTED&skoid=REDACTED&spr=REDACTED&sks=REDACTED&skt=REDACT

12
